In [1]:
import psycopg2
import pandas as pd
import numpy as np

connection = psycopg2.connect(host="savable-database.cq3phxuo3rta.ap-northeast-2.rds.amazonaws.com", dbname="savable_db", user="postgres", password="hundred100", port=5432)
# connection = psycopg2.connect(host="savable-database.cq3phxuo3rta.ap-northeast-2.rds.amazonaws.com", dbname=savable_db, user=postgres, password=hundred100, port=5432)
# host = endpoint
# user = 마스터 사용자의 이름

In [2]:
cur = connection.cursor() # cursor라는 인스턴스를 만들어서 그 인스턴스를 통해 데이터 조작을 진행

In [3]:
# sql문
query="SELECT cert_data->>'date' AS date, cert_data->>'check' as CHECK, cert_data->>'image' as image, data_row->>'challenge_id' as challenge_id FROM member m , jsonb_array_elements(m.certification) AS data_row,jsonb_array_elements(data_row->'cert') AS cert_data;"
# query="select * from member"
cur.execute(query)

In [4]:
rows = cur.fetchall()
for i in range(3):
  print(rows[i])

('2023-07-14T13:35:00.778+09', 'PASS', 'https://talk.kakaocdn.net/dna/jZjjO/bl56xYLfH0R/yKcoyJmmhNHLkhMwaaV1HT/i_f3fe1b67a1e2.jpg?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1783917292&signature=edqbt45sRthFgrLI70X96l7UXQs%3D', '1')
('2023-07-18T09:10:37.274+09', 'PASS', 'https://talk.kakaocdn.net/dna/bpSLyb/bl57nreDKVP/0Z9RygeLdKho5iEzsEWW6y/i_471e45558c1a.jpg?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1784247026&signature=%2FySI7KII2ee90mTIVevVgGd%2Fiic%3D', '1')
('2023-07-18T12:52:52.297+09', 'PASS', 'https://talk.kakaocdn.net/dna/3GIxK/bl57nooHc28/BvL2NMUp9VOfCQVFHqeOqY/i_4426ee1d418d.heic?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1784260367&signature=AzR3%2FDRLfPMsijzMwLemh4UqPU8%3D', '1')


In [5]:
df=pd.DataFrame(rows,columns=['date','check','image','challenge_id'])
df.head()

,date,check,image,challenge_id
0,2023-07-14T13:35:00.778+09,PASS,https://talk.kakaocdn.net/dna/jZjjO/bl56xYLfH0...,1
1,2023-07-18T09:10:37.274+09,PASS,https://talk.kakaocdn.net/dna/bpSLyb/bl57nreDK...,1
2,2023-07-18T12:52:52.297+09,PASS,https://talk.kakaocdn.net/dna/3GIxK/bl57nooHc2...,1
3,2023-07-19T08:32:29.376+09,PASS,https://talk.kakaocdn.net/dna/cA70ZM/bl56Sbjpn...,1
4,2023-07-20T08:25:49.358+09,PASS,https://talk.kakaocdn.net/dna/V7rc5/bl57nqfQ3M...,1


In [6]:
print(len(df))
cur.close()
connection.close()

6476


In [7]:
result = df.groupby(['challenge_id', 'check']).size().reset_index(name='count')
result

,challenge_id,check,count
0,1,FAIL,10
1,1,PASS,2058
2,1,UNCHECKED,1262
3,2,FAIL,4
4,2,PASS,306
5,2,UNCHECKED,349
6,3,FAIL,3
7,3,PASS,192
8,3,UNCHECKED,109
9,4,FAIL,1


In [8]:
delivery_df = df[df['challenge_id'].isin(['2', '4'])]  # 배달비 절약 챌린지
# selected_df = df[(df['challenge_id']=='1') | (df['challenge_id']=='2')]
len(delivery_df)
delivery_df.head()

,date,check,image,challenge_id
29,2023-07-31T20:05:18.259+09,FAIL,https://talk.kakaocdn.net/dna/ja4Ab/bl58VCeJRd...,2
30,2023-08-01T17:37:05.856+09,FAIL,https://talk.kakaocdn.net/dna/eHclQE/bl58rQx7P...,2
77,2023-09-07T16:36:39.962+09,FAIL,https://talk.kakaocdn.net/dna/RDerk/bl6eEk0xut...,4
229,2023-11-01T02:53:47.270+09,UNCHECKED,https://talk.kakaocdn.net/dna/jEmx3/bl6m90MMzR...,4
230,2023-11-05T17:43:27.850+09,UNCHECKED,https://talk.kakaocdn.net/dna/cJFsif/bl6nUY1Kb...,4


In [9]:
import requests
import os
image_urls=delivery_df.image
# default_path=' C:/Users/irene/PycharmProjects/downloadImage/delivery_challenge'
# print(os.path.join(default_path, 'delivery_challenge'))
# default_path='C:\Users\irene\PycharmProjects\downloadImage\venv\delivery_challenge'
default_path='venv/delivery_challenge'

683

In [10]:
for i,url in enumerate(image_urls):
    response = requests.get(url)
    if response.status_code == 200:
        # 이미지를 'downloaded_images' 폴더에 저장
        with open(default_path+'/'+str(i)+'*'+f'{hash(url)}.jpg', 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download image from {url}")